In [1]:
import sys
import pandas as pd
import numpy as np
sys.path.append('../src/')

In [2]:
APPLIANCE = ['fridge', 'dish_washer', 'washing_machine']
THRESHOLD = [50., 10., 20.]
MIN_ON = [1., 30., 30.]
MIN_OFF = [1., 3., 30.]

METER = 'aggregate'
SEQ_LEN = 60*8
BORDER = 16
BATCH_SIZE = 32

MAX_POWER = 2000

### UKDale training network

In [3]:
from utils.preprocessing import get_status
houses=[1,2,5]
ds_meter = []
ds_appliance = []
ds_status = []
for i in houses:
    ds = pd.read_feather('../data/ukdale/feather_files/UKDALE_%d_train.feather' %(i))
    ds.set_index('datetime', inplace=True)
  
    meter = ds[METER]
    appliances = ds[APPLIANCE]
    
    status = pd.DataFrame()
    for a in range(len(APPLIANCE)):
        status = pd.concat([status, get_status(ds[APPLIANCE[a]], THRESHOLD[a], MIN_OFF[a], MIN_ON[a])], axis=1)
    
    ds_meter.append(meter)
    ds_appliance.append(appliances)
    ds_status.append(status)

ds_len = [len(ds_meter[i]) for i in range(3)]

In [4]:
import torch
from torch.utils.data import Dataset, DataLoader
from utils.preprocessing import Power

ds_house_train = [Power(ds_meter[i][:int(0.8*ds_len[i])], 
                        ds_appliance[i][:int(0.8*ds_len[i])], 
                        ds_status[i][:int(0.8*ds_len[i])], 
                        SEQ_LEN, BORDER, MAX_POWER, True) for i in range(3)]

ds_house_valid = [Power(ds_meter[i][int(0.8*ds_len[i]):], 
                        ds_appliance[i][int(0.8*ds_len[i]):],
                        ds_status[i][int(0.8*ds_len[i]):], 
                        SEQ_LEN, BORDER, MAX_POWER, False) for i in range(3)]

ds_house_total  = [Power(ds_meter[i], ds_appliance[i], ds_status[i], 
                         SEQ_LEN, BORDER, MAX_POWER, False) for i in range(3)]

ds_train = torch.utils.data.ConcatDataset([ds_house_train[0], ds_house_train[2]])
ds_valid = torch.utils.data.ConcatDataset([ds_house_valid[0], ds_house_valid[2]])

dl_train = DataLoader(dataset = ds_train, batch_size = BATCH_SIZE, shuffle=True)
dl_valid = DataLoader(dataset = ds_valid, batch_size = BATCH_SIZE, shuffle=False)
dl_test = DataLoader(dataset = ds_house_total[1], batch_size = BATCH_SIZE, shuffle=False)

dl_house_test = [DataLoader(dataset = ds_house_total[1], batch_size = 1, shuffle=False)]
dl_house_valid = [DataLoader(dataset = ds_house_valid[i], batch_size = 1, shuffle=False) for i in [0,2]]
dl_house_total = [DataLoader(dataset = ds_house_total[i], batch_size = 1, shuffle=True) for i in range(3)]

In [5]:
import torch.nn as nn
import torch.optim as optim
from utils.net import PTPNet
from utils.training import train_model

batch_size = BATCH_SIZE
n_epochs = 100

train_loader = dl_train
valid_loader = dl_valid
test_loader = dl_test
model = PTPNet(1,3,32)
optimizer = optim.Adam(model.parameters(), lr=1.E-4)
criterion = nn.BCEWithLogitsLoss()
fn='../data/ukdale/network_weights/UKDALE_network_weights.pth'
model, train_loss, valid_loss, test_loss = train_model(model, batch_size, n_epochs, fn, train_loader, valid_loader, test_loader, optimizer, criterion)


[  1/100] train_loss: 0.56640 valid_loss: 0.45178 test_loss: 0.39688 
Validation loss decreased (inf --> 0.451783).  Saving model ...
[  2/100] train_loss: 0.38577 valid_loss: 0.34087 test_loss: 0.29473 
Validation loss decreased (0.451783 --> 0.340872).  Saving model ...
[  3/100] train_loss: 0.30618 valid_loss: 0.28832 test_loss: 0.24762 
Validation loss decreased (0.340872 --> 0.288317).  Saving model ...
[  4/100] train_loss: 0.26508 valid_loss: 0.25632 test_loss: 0.21095 
Validation loss decreased (0.288317 --> 0.256318).  Saving model ...
[  5/100] train_loss: 0.23441 valid_loss: 0.23656 test_loss: 0.18588 
Validation loss decreased (0.256318 --> 0.236562).  Saving model ...
[  6/100] train_loss: 0.21302 valid_loss: 0.22298 test_loss: 0.17206 
Validation loss decreased (0.236562 --> 0.222982).  Saving model ...
[  7/100] train_loss: 0.19793 valid_loss: 0.20521 test_loss: 0.15555 
Validation loss decreased (0.222982 --> 0.205215).  Saving model ...
[  8/100] train_loss: 0.18838 va

### REFIT training network

##### Fridge case

In [6]:
APPLIANCE = ['fridge']
THRESHOLD = [50.]
MIN_ON = [1.]
MIN_OFF = [1.]

METER = 'aggregate'
SEQ_LEN = 60*8
BORDER = 16
BATCH_SIZE = 32

MAX_POWER = 2000.

In [7]:
from utils.preprocessing import get_status
original_meter = []
ds_appliance = []
ds_status = []
houses=[2,5,9,12,15]
for i in houses:
    ds = pd.read_feather('../data/refit/feather_files/fridge/REFIT_%d.feather' %i)
    ds.set_index('datetime', inplace=True)
    
    meter = ds[METER]
    appliances = ds[APPLIANCE]
    
    status = pd.DataFrame()
    for a in range(len(APPLIANCE)):
        status = pd.concat([status, get_status(ds[APPLIANCE[a]], THRESHOLD[a], MIN_OFF[a], MIN_ON[a])], axis=1)
    
    original_meter.append(meter)
    ds_appliance.append(appliances)
    ds_status.append(status)

ds_len = [len(original_meter[i]) for i in range(5)]

In [8]:
import torch
from torch.utils.data import Dataset, DataLoader
from utils.preprocessing import Power

ds_house_total  = [Power(original_meter[i], ds_appliance[i], ds_status[i],SEQ_LEN, BORDER, MAX_POWER, False) for i in range(5)]

ds_train      = torch.utils.data.ConcatDataset([ds_house_total[0], 
                                                ds_house_total[1], 
                                                ds_house_total[2]])
ds_valid       = torch.utils.data.ConcatDataset([ds_house_total[3]])

dl_train = DataLoader(dataset = ds_train, batch_size = BATCH_SIZE, shuffle=True)
dl_valid = DataLoader(dataset = ds_valid, batch_size = BATCH_SIZE, shuffle=False)
dl_test = DataLoader(dataset = ds_house_total[4], batch_size = BATCH_SIZE, shuffle=False)

dl_house_test = [DataLoader(dataset = ds_house_total[4], batch_size = 1, shuffle=False)]
dl_house_valid = [DataLoader(dataset = ds_house_total[3], batch_size = 1, shuffle=False)]
dl_house_total = [DataLoader(dataset = ds_house_total[i], batch_size = 1, shuffle=False) for i in range(5)]


In [9]:
import torch.nn as nn
import torch.optim as optim
from utils.net import PTPNet
from utils.training import train_model

batch_size = BATCH_SIZE
n_epochs = 100

train_loader = dl_train
valid_loader = dl_valid
test_loader = dl_test

model = PTPNet(1,1,32)
optimizer = optim.Adam(model.parameters(), lr=1.E-4)
criterion = nn.BCEWithLogitsLoss()
fn = '../data/refit/network_weights/REFIT_fridge_network_weights.pth'
model, train_loss, valid_loss, test_loss = train_model(model, batch_size, n_epochs, fn, train_loader, valid_loader, test_loader, optimizer, criterion)

[  1/100] train_loss: 0.60252 valid_loss: 0.61343 test_loss: 0.54161 
Validation loss decreased (inf --> 0.613428).  Saving model ...
[  2/100] train_loss: 0.51198 valid_loss: 0.58967 test_loss: 0.44051 
Validation loss decreased (0.613428 --> 0.589669).  Saving model ...
[  3/100] train_loss: 0.47456 valid_loss: 0.62230 test_loss: 0.45843 
[  4/100] train_loss: 0.45394 valid_loss: 0.60215 test_loss: 0.46900 
[  5/100] train_loss: 0.44118 valid_loss: 0.67261 test_loss: 0.58120 
[  6/100] train_loss: 0.42975 valid_loss: 0.66753 test_loss: 0.57120 
[  7/100] train_loss: 0.41930 valid_loss: 0.62104 test_loss: 0.47636 
[  8/100] train_loss: 0.41206 valid_loss: 0.69335 test_loss: 0.58430 
[  9/100] train_loss: 0.40404 valid_loss: 0.68225 test_loss: 0.54839 
[ 10/100] train_loss: 0.39878 valid_loss: 0.76673 test_loss: 0.66672 
[ 11/100] train_loss: 0.39137 valid_loss: 0.76341 test_loss: 0.57578 
[ 12/100] train_loss: 0.38679 valid_loss: 0.84663 test_loss: 0.63659 
[ 13/100] train_loss: 0.382

##### Dishwasher case

In [2]:
APPLIANCE = ['dishwasher']
THRESHOLD = [100.]
MIN_ON = [30.]
MIN_OFF = [90.]

METER = 'aggregate'
SEQ_LEN = 60*8
BORDER = 16
BATCH_SIZE = 32

MAX_POWER = 2000.

In [3]:
from utils.preprocessing import get_status
original_meter = []
ds_appliance = []
ds_status = []
houses=[5,7,9,13,16,18,20]

for i in houses:
    ds = pd.read_feather('../data/refit/feather_files/dishwasher/REFIT_%d.feather' %i)
    ds.set_index('datetime', inplace=True)
    
    meter = ds[METER]
    appliances = ds[APPLIANCE]
    
    status = pd.DataFrame()
    for a in range(len(APPLIANCE)):
        status = pd.concat([status, get_status(ds[APPLIANCE[a]], THRESHOLD[a], MIN_OFF[a], MIN_ON[a])], axis=1)
    
    original_meter.append(meter)
    ds_appliance.append(appliances)
    ds_status.append(status)

ds_len = [len(original_meter[i]) for i in range(len(houses))]

In [4]:
import torch
from torch.utils.data import Dataset, DataLoader
from utils.preprocessing import Power

ds_house_total  = [Power(original_meter[i], ds_appliance[i], ds_status[i],SEQ_LEN, BORDER, MAX_POWER, False) for i in range(7)]
ds_train      = torch.utils.data.ConcatDataset([ds_house_total[i] for i in range(5)])
ds_valid       = torch.utils.data.ConcatDataset([ds_house_total[5]])

dl_train = DataLoader(dataset = ds_train, batch_size = BATCH_SIZE, shuffle=False)
dl_valid = DataLoader(dataset = ds_valid, batch_size = BATCH_SIZE, shuffle=False)
dl_test = DataLoader(dataset = ds_house_total[6], batch_size = BATCH_SIZE, shuffle=False)

dl_house_total = [DataLoader(dataset = ds_house_total[i], batch_size = 1, shuffle=False) for i in range(7)]

In [5]:
import torch.nn as nn
import torch.optim as optim
from utils.net import PTPNet
from utils.training import train_model

batch_size = BATCH_SIZE
n_epochs = 100

train_loader = dl_train
valid_loader = dl_valid
test_loader = dl_test

model = PTPNet(1,1,32)
optimizer = optim.Adam(model.parameters(), lr=1.E-4)
criterion = nn.BCEWithLogitsLoss()
fn = '../data/refit/network_weights/REFIT_dishwasher_network_weights.pth'
model, train_loss, valid_loss, test_loss = train_model(model, batch_size, n_epochs, fn, train_loader, valid_loader, test_loader, optimizer, criterion)

[  1/100] train_loss: 0.23210 valid_loss: 0.06540 test_loss: 0.05800 
Validation loss decreased (inf --> 0.065397).  Saving model ...
[  2/100] train_loss: 0.11197 valid_loss: 0.03870 test_loss: 0.03439 
Validation loss decreased (0.065397 --> 0.038697).  Saving model ...
[  3/100] train_loss: 0.09023 valid_loss: 0.04099 test_loss: 0.03088 
[  4/100] train_loss: 0.08082 valid_loss: 0.04284 test_loss: 0.02929 
[  5/100] train_loss: 0.07443 valid_loss: 0.04346 test_loss: 0.02831 
[  6/100] train_loss: 0.07041 valid_loss: 0.04546 test_loss: 0.02919 
[  7/100] train_loss: 0.06738 valid_loss: 0.04658 test_loss: 0.02917 
[  8/100] train_loss: 0.06441 valid_loss: 0.04759 test_loss: 0.03028 
[  9/100] train_loss: 0.06195 valid_loss: 0.04972 test_loss: 0.03037 
[ 10/100] train_loss: 0.05985 valid_loss: 0.04843 test_loss: 0.02981 
[ 11/100] train_loss: 0.05782 valid_loss: 0.05004 test_loss: 0.03010 
[ 12/100] train_loss: 0.05621 valid_loss: 0.05071 test_loss: 0.02990 
[ 13/100] train_loss: 0.055

##### Washing-machine case

In [7]:
APPLIANCE = ['washing_machine']
THRESHOLD = [30.]
MIN_ON = [30.]
MIN_OFF = [3.]

METER = 'aggregate'
SEQ_LEN = 60*8
BORDER = 16
BATCH_SIZE = 32

MAX_POWER = 2000.

In [8]:
from utils.preprocessing import get_status
original_meter = []
ds_appliance = []
ds_status = []
houses = [2,5,7,9,15,16,17,18,8]

for i in houses:
    ds = pd.read_feather('../data/refit/feather_files/washing_machine/REFIT_%d.feather' %i)
    ds.set_index('datetime', inplace=True)
    
    meter = ds[METER]
    appliances = ds[APPLIANCE]
    
    status = pd.DataFrame()
    for a in range(len(APPLIANCE)):
        status = pd.concat([status, get_status(ds[APPLIANCE[a]], THRESHOLD[a], MIN_OFF[a], MIN_ON[a])], axis=1)
    
    original_meter.append(meter)
    ds_appliance.append(appliances)
    ds_status.append(status)

ds_len = [len(original_meter[i]) for i in range(len(houses))]

In [9]:
ds_house_total  = [Power(original_meter[i], ds_appliance[i], ds_status[i], SEQ_LEN, BORDER, MAX_POWER, True) for i in range(7)]
ds_house_total.append(Power(original_meter[7], ds_appliance[7], ds_status[7], SEQ_LEN, BORDER, MAX_POWER, False))
ds_house_total.append(Power(original_meter[8], ds_appliance[8], ds_status[8], SEQ_LEN, BORDER, MAX_POWER, False))

ds_train      = torch.utils.data.ConcatDataset([ds_house_total[0], 
                                                ds_house_total[1], 
                                                ds_house_total[2],
                                                ds_house_total[3],
                                                ds_house_total[4],
                                                ds_house_total[5],
                                                ds_house_total[6]])
ds_valid       = torch.utils.data.ConcatDataset([ds_house_total[7]])

dl_train = DataLoader(dataset = ds_train, batch_size = BATCH_SIZE, shuffle=False)
dl_valid = DataLoader(dataset = ds_valid, batch_size = BATCH_SIZE, shuffle=False)
dl_test = DataLoader(dataset = ds_house_total[8], batch_size = BATCH_SIZE, shuffle=False)

dl_house_test = [DataLoader(dataset = ds_house_total[8], batch_size = 1, shuffle=False)]
dl_house_valid = [DataLoader(dataset = ds_house_total[7], batch_size = 1, shuffle=False)]
dl_house_total = [DataLoader(dataset = ds_house_total[i], batch_size = 1, shuffle=False) for i in range(9)]

In [10]:
import torch.nn as nn
import torch.optim as optim
from utils.net import PTPNet
from utils.training import train_model

batch_size = BATCH_SIZE
n_epochs = 100

train_loader = dl_train
valid_loader = dl_valid
test_loader = dl_test

model = PTPNet(1,1,32)
optimizer = optim.Adam(model.parameters(), lr=1.E-4)
criterion = nn.BCEWithLogitsLoss()
fn = '../data/refit/network_weights/REFIT_washingmachine_network_weights.pth'
model, train_loss, valid_loss, test_loss = train_model(model, batch_size, n_epochs, fn, train_loader, valid_loader, test_loader, optimizer, criterion)

[  1/100] train_loss: 0.18894 valid_loss: 0.05952 test_loss: 0.09366 
Validation loss decreased (inf --> 0.059522).  Saving model ...
[  2/100] train_loss: 0.10294 valid_loss: 0.09783 test_loss: 0.10452 
[  3/100] train_loss: 0.09155 valid_loss: 0.10587 test_loss: 0.09763 
[  4/100] train_loss: 0.08521 valid_loss: 0.07743 test_loss: 0.06716 
[  5/100] train_loss: 0.07977 valid_loss: 0.05774 test_loss: 0.06596 
Validation loss decreased (0.059522 --> 0.057741).  Saving model ...
[  6/100] train_loss: 0.07846 valid_loss: 0.05603 test_loss: 0.06466 
Validation loss decreased (0.057741 --> 0.056030).  Saving model ...
[  7/100] train_loss: 0.07566 valid_loss: 0.04998 test_loss: 0.05833 
Validation loss decreased (0.056030 --> 0.049977).  Saving model ...
[  8/100] train_loss: 0.07505 valid_loss: 0.06249 test_loss: 0.06172 
[  9/100] train_loss: 0.07510 valid_loss: 0.04661 test_loss: 0.05857 
Validation loss decreased (0.049977 --> 0.046607).  Saving model ...
[ 10/100] train_loss: 0.07461 